In [1]:
#!pip install pycryptodome

In [2]:
from sage.all import *

In [3]:
from sage.all import *
import hashlib
import hmac
import os

class KEMElGamal:
    def __init__(self, security_parameter):
        self.security_parameter = security_parameter
        self.generate_keys()

    def generate_keys(self):
        p = random_prime(2 ** self.security_parameter)
        g = primitive_root(p)
        x = randint(1, p - 2)
        h = pow(g, x, p)

        self.private_key = x
        self.public_key = (p, g, h)

    def encapsulate(self, public_key):
        p, g, h = public_key
        r = randint(1, p - 2)
        c1 = pow(g, r, p)
        c2 = pow(h, r, p)

        key_seed = hashlib.sha256()
        key_seed.update(str(c2).encode())
        key = key_seed.digest()

        return (c1, c2), key

    def decapsulate(self, ciphertext):
        c1, c2 = ciphertext
        shared_secret = pow(c1, self.private_key, self.public_key[0])

        key_seed = hashlib.sha256()
        key_seed.update(str(shared_secret).encode())
        key = key_seed.digest()

        return key

class PKEFujisakiOkamoto(KEMElGamal):
    def __init__(self, security_parameter):
        super().__init__(security_parameter)

    def encrypt(self, plaintext, public_key):
        kem_ciphertext, key = self.encapsulate(public_key)
        r = os.urandom(32)
        hmac_key = hmac.new(key, r, hashlib.sha256).digest()

        ciphertext = int.from_bytes(plaintext, byteorder='big') ^ int.from_bytes(hmac_key, byteorder='big')
        return kem_ciphertext, (r, ciphertext.to_bytes((ciphertext.bit_length() + 7) // 8, byteorder='big'))

    def decrypt(self, ciphertext):
        kem_ciphertext, (r, encrypted_data) = ciphertext
        key = self.decapsulate(kem_ciphertext)
        hmac_key = hmac.new(key, r, hashlib.sha256).digest()

        decrypted_data = int.from_bytes(encrypted_data, byteorder='big') ^ int.from_bytes(hmac_key, byteorder='big')
        return decrypted_data.to_bytes((decrypted_data.bit_length() + 7) // 8, byteorder='big')

In [4]:
def main():
    security_parameter = 128

    # Cria uma instância do PKE Fujisaki-Okamoto
    pke = PKEFujisakiOkamoto(security_parameter)

    # Obtenha as chaves pública e privada
    public_key = pke.public_key
    private_key = pke.private_key

    # Mensagem original
    plaintext = b"Estruturas Criptograficas - Grupo 17!"

    # Criptografa a mensagem usando a chave pública
    ciphertext = pke.encrypt(plaintext, public_key)
    print("Ciphertext:", ciphertext)

    # Descriptografa a mensagem usando a chave privada
    decrypted_text = pke.decrypt(ciphertext)
    print("Decrypted text:", decrypted_text)

    assert plaintext == decrypted_text, "A mensagem descriptografada não corresponde à mensagem original"

if __name__ == "__main__":
    main()

Ciphertext: ((219664560206525943559094038716465720292, 174031781362970832805351804137357984439), (b'M\xb3\x04z\xc96\x16\xc7\xc9>y\xfa\xb3\x8c\xcc\xcc\xbf\xde\x83\x0c\xb8\x89\x98\x15\x18Bv\x0e\xa5\xadG\x13', b'Estru\xb3MR\xe6\xb1\x19\xb2\x95\xeb\x07\xdb5/\xf8\xb1\x8c\xe7\xcb\xf0)\xec\xd6\x8d\x91W\xa5\xcc\xa7\xd1\xd4\xce\x87'))
Decrypted text: b'Estruturas Criptograficas - Grupo 17!'
